In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import ray
from prophet import Prophet

Preliminary data changes

In [20]:
df = pd.read_csv("product_sales.csv")
# Drop random columns
df = df.drop(columns=["Unnamed: 107", "94"])
# Melt data and remove strings for week names
df = df.melt(id_vars=["Scode", "Pcode", "Price"], var_name="Week", value_name="Quantity Sold")
df["Week"] = df["Week"].str.extract('(\d+)').astype(int)

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
/var/folders/hh/rn_fsj015zz6cmlmssx4w0y00000gn/T/ipykernel_24642/2419203732.py:6: SyntaxWarning: invalid escape sequence '\d'
  df["Week"] = df["Week"].str.extract('(\d+)').astype(int)


Interpolate data: for each date present in the dataset, we create a data point for each SKU when it doesn’t exist– it will use the same quantity and unit price as the most recent date where that SKU has a data point. 
Use time series (ARIMA model) to forecast future dates. Use these as validation data for the hyperparameter search. This way we are making optimal policies for the most recent data, and calculating our loss based on forecasted data.


# Use Prophet for short term interpolation

In [21]:
df

,Scode,Pcode,Price,Week,Quantity Sold
0,Store1,SKU1,$24.00,0,23
1,Store1,SKU2,$63.00,0,23
2,Store1,SKU3,$93.00,0,7
3,Store1,SKU4,$48.00,0,50
4,Store1,SKU5,$62.00,0,8
...,...,...,...,...,...
623995,Store5,SKU1809,$71.00,103,39
623996,Store5,SKU1563,$64.00,103,24
623997,Store5,SKU897,$77.00,103,28
623998,Store5,SKU1142,$81.00,103,31


In [22]:
# Will contain (store, sku) pair
all_unique_combos = []
stores = df["Scode"].unique()
# How many stores do we want to forecast for
stores = stores[:1] 
for store in stores:
    holder = df[df["Scode"] == store]
    skus = holder["Pcode"].unique()
    for sku in skus:
        all_unique_combos.append((store, sku))


In [23]:
# Parellelize this forecasting for speed
@ray.remote
def forecast_dates(df, pair):
    value_column = df[(df["Pcode"] == pair[1]) & (df["Scode"] == pair[0])].reset_index(drop=True)
    value_column = value_column["Quantity Sold"]
    start_date = "2023-01-01"  # Choose an arbitrary start date
    date_range = pd.date_range(start=start_date, periods=len(value_column), freq='W') 
    data = pd.DataFrame({'ds': date_range, 'y': value_column})
    model = Prophet(weekly_seasonality=True, yearly_seasonality=True)
    model.fit(data)
    future = model.make_future_dataframe(periods=50, freq='W')
    forecast = model.predict(future)
    return data, forecast

In [24]:
ray.init(log_to_driver=False, ignore_reinit_error=True)
ds = ray.data.from_pandas(df).repartition(1)
block_refs = ds.get_internal_block_refs()


2024-12-01 09:41:07,966	INFO worker.py:1652 -- Calling ray.init() again after it has already been called.
2024-12-01 09:41:08,259	WARNING dataset.py:4895 -- `Dataset.get_internal_block_refs()` is deprecated. Use `Dataset.iter_internal_ref_bundles()` instead.
2024-12-01 09:41:08,261	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-01_09-39-21_327780_24642/logs/ray-data
2024-12-01 09:41:08,262	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> AllToAllOperator[Repartition]


Running 0: 0.00 row [00:00, ? row/s]

- Repartition 1: 0.00 row [00:00, ? row/s]

Split Repartition 2:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

In [25]:
block_refs[0]

ObjectRef(842a8fe6170509b1ffffffffffffffffffffffff0100000001000000)

In [26]:
forecast_futures = [
    forecast_dates.remote(
        block_refs[0],
        pair
    )
    for pair in all_unique_combos
]

In [27]:
results = ray.get(forecast_futures)

In [28]:
store1df = df[df["Scode"] == "Store1"]

In [29]:
store1df[store1df["Pcode"] == "SKU1"].reset_index(drop=True)

,Scode,Pcode,Price,Week,Quantity Sold
0,Store1,SKU1,$24.00,0,23
1,Store1,SKU1,$24.00,1,48
2,Store1,SKU1,$24.00,2,47
3,Store1,SKU1,$24.00,3,42
4,Store1,SKU1,$24.00,4,22
...,...,...,...,...,...
99,Store1,SKU1,$24.00,99,133
100,Store1,SKU1,$24.00,100,229
101,Store1,SKU1,$24.00,101,381
102,Store1,SKU1,$24.00,102,21


In [30]:
subset = store1df[(store1df["Scode"] == "Store1") & (store1df["Pcode"] == "SKU1")].iloc[0]

In [31]:
subset

Scode             Store1
Pcode               SKU1
Price            $24.00 
Week                   0
Quantity Sold         23
Name: 0, dtype: object

In [32]:
# start with all info from store1
skus = store1df["Pcode"].unique()
store = "Store1"
for i,sku in enumerate(skus):
    # SKU info
    subset = store1df[(store1df["Scode"] == store) & (store1df["Pcode"] == sku)].iloc[0]
    price = subset["Price"]
    # Get new week info
    curr_df = results[i][1]
    new_df = curr_df[104:].reset_index()
    new_df = new_df[["index", "yhat"]].rename(columns={"index": "Week", "yhat": "Quantity Sold"})
    new_df["Scode"] = store
    new_df["Pcode"] = sku
    new_df["Price"] = price
    store1df = pd.concat([store1df, new_df]).reset_index(drop=True)

In [35]:
store1df = store1df.drop_duplicates()

In [36]:
store1df.to_csv("ForecastedInformation.csv", index=False)